In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from scipy.sparse import csr_matrix, save_npz
from snnlib import matrix, cos_knn_index
from sklearn.preprocessing import normalize

In [3]:
def np2csr(fname):
    data = np.load(fname)
    if type(data) is np.ndarray:
        return csr_matrix(data)
    ind, val, ptr, shape = data['indices'], data['data'], data['indptr'], data['shape']
    return csr_matrix((val, ind, ptr), shape)

In [4]:
%%time
d = np2csr('data/GSE84133_X.npy')

CPU times: user 87.2 ms, sys: 135 ms, total: 222 ms
Wall time: 228 ms


In [5]:
%%time
index = cos_knn_index(d)
knng = index.knng(10, True)

CPU times: user 1.24 s, sys: 0 ns, total: 1.24 s
Wall time: 166 ms


In [6]:
graph = np2csr('data/knnga_1M_neuron_10_1_0.7_t.npz')

In [7]:
graph.shape, graph.nnz, graph.nnz/(graph.shape[0]*graph.shape[1])

((1306127, 1306127), 13061270, 7.65622332284686e-06)

In [8]:
# rows with close neighbors
hsim = [(i, graph.indices[j], graph.data[j]) for i,j in enumerate(graph.indptr[:-1]) if graph.data[j] > 0.9 ]
rids = list(set([i[0] for i in hsim]))
print(len(rids))
for i in range(min(10, len(rids))):
    print(rids[i], graph[rids[i]])
    print()

15232
1081344   (0, 871553)	0.9176462
  (0, 1010115)	0.917061
  (0, 835241)	0.91468596
  (0, 1146693)	0.9145409
  (0, 1223686)	0.9120873
  (0, 732942)	0.91109043
  (0, 854060)	0.90774715
  (0, 1228656)	0.9070368
  (0, 670113)	0.90593845
  (0, 1091010)	0.90540576

32769   (0, 1010115)	0.9516853
  (0, 835241)	0.9499493
  (0, 871553)	0.9465259
  (0, 1228656)	0.94560474
  (0, 231746)	0.9429833
  (0, 1146693)	0.9409759
  (0, 1211053)	0.9386376
  (0, 849790)	0.93837726
  (0, 67990)	0.9382142
  (0, 732942)	0.9381108

950273   (0, 1010115)	0.95828277
  (0, 769320)	0.9525164
  (0, 1048165)	0.95094603
  (0, 1303642)	0.95070046
  (0, 1027553)	0.9503209
  (0, 736019)	0.9488457
  (0, 724496)	0.9461821
  (0, 732942)	0.9457748
  (0, 739205)	0.94553965
  (0, 871553)	0.9453147

851971   (0, 1010115)	0.98659474
  (0, 835241)	0.9811012
  (0, 871553)	0.9773496
  (0, 1303642)	0.97511584
  (0, 984348)	0.97475415
  (0, 1146693)	0.9741656
  (0, 1022905)	0.973939
  (0, 37873)	0.97373664
  (0, 1027553)	0.973215

In [9]:
# rows with far away neighbors
hsim = [(i, graph.indices[j], graph.data[j]) for i,j in enumerate(graph.indptr[:-1]) if graph.data[j] < 0.2 ]
rids = list(set([i[0] for i in hsim]))
print(len(rids))
for i in range(min(10, len(rids))):
    print(rids[i], graph[rids[i]])
    print()

235
1186306   (0, 591529)	0.19948532
  (0, 424924)	0.18729432
  (0, 311824)	0.1823288
  (0, 326236)	0.18101029
  (0, 560028)	0.18086566
  (0, 578296)	0.1796955
  (0, 508949)	0.17953274
  (0, 333513)	0.1784079
  (0, 527180)	0.17789239
  (0, 486468)	0.17681895

1200130   (0, 560028)	0.18009849
  (0, 578296)	0.18007423
  (0, 307859)	0.1777303
  (0, 591529)	0.17576602
  (0, 517752)	0.1752662
  (0, 311824)	0.17397235
  (0, 424924)	0.17213121
  (0, 572508)	0.17112151
  (0, 648680)	0.16911054
  (0, 533489)	0.16875376

381959   (0, 578296)	0.19893396
  (0, 560028)	0.194948
  (0, 307859)	0.19483793
  (0, 115450)	0.18697488
  (0, 648680)	0.1853028
  (0, 591529)	0.18493664
  (0, 378350)	0.18490018
  (0, 590069)	0.1846177
  (0, 572508)	0.1836178
  (0, 652571)	0.18231063

893447   (0, 832145)	0.18684182
  (0, 440449)	0.1860948
  (0, 157758)	0.18290018
  (0, 560028)	0.1826267
  (0, 265137)	0.18258105
  (0, 146754)	0.17992146
  (0, 578296)	0.17905894
  (0, 765131)	0.17854957
  (0, 187310)	0.17842178


In [10]:
a = index.get_index_data()
m = np.sqrt(a.multiply(a).sum(1)).T
m[m<0.9999].tolist()[0]

[]

In [11]:
# create random csr_matrix with 1K rows, 10K columns, and a 20% fill factor
# matrix.random(nrows, ncols, factor=0.05)
m = matrix.random(10000, 5000, 0.15)
m_csr = m.as_csr()

In [12]:
%%time
# create an index from the matrix
index = cos_knn_index(m_csr)

CPU times: user 120 ms, sys: 507 ms, total: 627 ms
Wall time: 629 ms


In [13]:
%%time
# compute the nearest neighbor graph using the approx method
# index.knng(k, approx=false, cfactor=2.0, nenhance=1)
knng = index.knng(10, True, cfactor=2.0, nenhance=1, minprune=1.0, verb=True)

Number of rows: 10000
Number of columns: 5000
Number of non-zeros: 7495010
Maximum rows: 10000
Maximum non-zeroes: 7495010
k: 10
approximate: 1
cfactor: 2
nenhance: 1
minprune: 1
nthreads: 12
nbrs storage: 10000 x 20
ninitcands: 200000
ndotproducts: 296503
npruned: 0

CPU times: user 9.81 s, sys: 276 ms, total: 10.1 s
Wall time: 3.75 s


In [14]:
%%time
knngbf = index.knngbf(10, verb=True)

Number of rows: 10000
Number of columns: 5000
Number of non-zeros: 7495010
Maximum rows: 10000
Maximum non-zeroes: 7495010
k: 10
nthreads: 12
nbrs storage: 10000 x 10
CPU times: user 4min 1s, sys: 1.35 s, total: 4min 2s
Wall time: 28.4 s


In [15]:
def python_knng(mat, k):
    """ Compute the knng using standard python techniques """
    # compute the similarity matrix
    matn = normalize(mat, norm='l2', axis=1)
    sims = matn.dot(matn.T)
    # now restrict rows to top-k values
    nrows = sims.shape[0]
    ind, val, ptr = sims.indices, sims.data, sims.indptr
    nnz = 0
    for i in range(nrows):
        s = ptr[i]
        e = ptr[i+1]
        vals = sorted([p for p in zip(val[s:e], ind[s:e]) if p[1] != i], reverse=True)
        n = min(e-s, k)
        for j in range(n):
            val[nnz+j] = vals[j][0]
            ind[nnz+j] = vals[j][1]
        ptr[i] = nnz
        nnz += n
    ptr[nrows] = nnz
    return csr_matrix((val[:nnz], ind[:nnz], ptr[:]))   

In [16]:
%%time 
gtknng = python_knng(m_csr, 10)

CPU times: user 6min 50s, sys: 2.73 s, total: 6min 53s
Wall time: 6min 54s


In [17]:
def csr_sort_nnzs_by_value(mat, reverse=False, copy=False):
    r""" Sort the nnzs in the matrix by value.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        s = ptr[i]
        e = ptr[i+1]
        vals = sorted(zip(val[s:e], ind[s:e]), reverse=reverse)
        for j in range(len(vals)):
            val[s+j] = vals[j][0]
            ind[s+j] = vals[j][1]
    return mat

def recall(gt, pr, eps=1e-3):
    """
    Compute the recall for a given (approximate) nearest neighbor graph.
    Elements at the edge of the neighborhood (within `eps` of the farthest neighbor)
    are accepted as valid.
    @param gt Ground truth graph
    @param pr Predicted graph
    @return recall, verrors (Value errors on matching indices, if any)
    """
    if gt.shape[0] != pr.shape[0] or gt.shape[0] != pr.shape[0]:
        raise ValueError("Graphs don't match in size.")
    # make sure graphs have sorted nnzs
    csr_sort_nnzs_by_value(gt, reverse=True)
    csr_sort_nnzs_by_value(pr, reverse=True)
        
    nrows = gt.shape[0]
    gind = gt.indices
    gval = gt.data
    gptr = gt.indptr
    pind = pr.indices
    pval = pr.data
    pptr = pr.indptr
    r = 0.0
    nr = nrows
    verrors = []
    for i in range(nrows):
        gs  = gptr[i]
        ge  = gptr[i+1]
        if ge == gs:
            # no values in this row
            nr -= 1
            continue
        ps  = pptr[i]
        pe  = pptr[i+1]
        mv  = gval[ge-1] # minimum value
        n = 0
        g   = gs
        p   = ps
        while p < pe and g < ge:
            gid = gind[g]
            gv  = gval[g]
            pid = pind[p]
            pv  = pval[p]
            if pid == gid:
                if abs(gv-pv) < eps:
                    n += 1
                else:
                    verrors.append((i, gid, gv, pv))
                g += 1
                p += 1
            elif abs(pv-mv) < eps:
                n += 1
                p += 1
            elif abs(pv-gv) < eps:
                n += 1
                p += 1
                g += 1
            elif pv < gv:
                g += 1
            else:
                p += 1
        r += n/(ge-gs)
    return r/nr, verrors

In [18]:
if 'gtknng' not in locals():
    gtknng = knngbf

r, verr = recall(gtknng, knng, eps=1e-5)
r, verr

(0.8121570198277502, [])

In [19]:
def cos(a, b):
    return a.dot(b.T).data[0]

if verr:
    matp = normalize(m_csr, norm='l2', axis=1)
    matc = index.get_index_data()
    q = verr[0][0]
    c = verr[0][1]
    print(q, c, cos(matp[q], matp[c]), cos(matc[q], matc[c]))
    print(sorted(zip(gtknng[q].data, gtknng[q].indices), reverse=True))
    print(sorted(zip(knng[q].data, knng[q].indices), reverse=True))

In [20]:
recall(gtknng, knngbf, eps=1e-5)

(1.0, [])

In [21]:
%%time
knng2 = index.knng(10, True, cfactor=2.0, nenhance=0, minprune=0.0, verb=True)

Number of rows: 10000
Number of columns: 5000
Number of non-zeros: 7495010
Maximum rows: 10000
Maximum non-zeroes: 7495010
k: 10
approximate: 1
cfactor: 2
nenhance: 0
minprune: 0
nthreads: 12
nbrs storage: 10000 x 20
ninitcands: 200000
ndotproducts: 100140
npruned: 60933

CPU times: user 2.88 s, sys: 78.9 ms, total: 2.96 s
Wall time: 845 ms


In [22]:
recall(gtknng, knng2)

(0.8144302022831874, [])

In [23]:
%%time
knng3 = index.knng(10, True, cfactor=2.0, nenhance=0, minprune=0.5, verb=True)

Number of rows: 10000
Number of columns: 5000
Number of non-zeros: 7495010
Maximum rows: 10000
Maximum non-zeroes: 7495010
k: 10
approximate: 1
cfactor: 2
nenhance: 0
minprune: 0.5
nthreads: 12
nbrs storage: 10000 x 20
ninitcands: 200000
ndotproducts: 199990
npruned: 10

CPU times: user 5.11 s, sys: 237 ms, total: 5.35 s
Wall time: 1.89 s


In [24]:
recall(gtknng, knng3)

(0.8144302022831874, [])

In [25]:
for mp in [0.1, 0.25, 0.5, 0.75, 1.0]:
    print(mp, recall(gtknng, index.knng(10, True, cfactor=2.0, nenhance=0, minprune=mp)))

0.1 (0.8144302022831874, [])
0.25 (0.8144302022831874, [])
0.5 (0.8144302022831874, [])
0.75 (0.8144302022831874, [])
1.0 (0.8144302022831874, [])


In [26]:
for cf in range(2,20,3):
    for ne in range(1,10,3):
        print(cf, ne, recall(gtknng, index.knng(10, True, cfactor=cf, nenhance=ne, minprune=0.7, verb=True)))

Number of rows: 10000
Number of columns: 5000
Number of non-zeros: 7495010
Maximum rows: 10000
Maximum non-zeroes: 7495010
k: 10
approximate: 1
cfactor: 2
nenhance: 1
minprune: 0.7
nthreads: 12
nbrs storage: 10000 x 20
ninitcands: 200000
ndotproducts: 286664
npruned: 20062

2 1 (0.8144402163028149, [])
Number of rows: 10000
Number of columns: 5000
Number of non-zeros: 7495010
Maximum rows: 10000
Maximum non-zeroes: 7495010
k: 10
approximate: 1
cfactor: 2
nenhance: 4
minprune: 0.7
nthreads: 12
nbrs storage: 10000 x 20
ninitcands: 200000
ndotproducts: 546656
npruned: 80248

2 4 (0.8144402163028149, [])
Number of rows: 10000
Number of columns: 5000
Number of non-zeros: 7495010
Maximum rows: 10000
Maximum non-zeroes: 7495010
k: 10
approximate: 1
cfactor: 2
nenhance: 7
minprune: 0.7
nthreads: 12
nbrs storage: 10000 x 20
ninitcands: 200000
ndotproducts: 806648
npruned: 140434

2 7 (0.8144402163028149, [])
Number of rows: 10000
Number of columns: 5000
Number of non-zeros: 7495010
Maximum rows